<a href="https://colab.research.google.com/github/suchy1713/InteligencjaObliczeniowa/blob/master/breast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROJEKT 1

piersi  
las  
skuteczność

pliczek (breast-cancer.data): https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/

TODO:
1. <s>przetwarzanie danych</s>
2. trening modelu
3. tuning parametrów (grid search + cross val) (przynajmniej dwa) (https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/)
4. confusion matrix wypisać
5. porównać wyniki gołych danych, znormalizowanych, zestandaryzowanych i przepuszczonych przez PCA z dwiema różnymi ilościami komponentów (5 różnych testów)

In [23]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
pd.set_option('display.max_columns', 500)

features = ['class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'deg-malig', 'breast', 'breast-quad', 'irradiat']

def missing_vals(df):
    cols = df.isna().sum()/df.shape[0]
    cols = pd.DataFrame({'Column name': cols.index, 'Missing %': cols.values})
    rows = (df.isna().sum(axis=1)/df.shape[1]).value_counts()
    rows = pd.DataFrame({'Missing %': rows.index, 'Quantity': rows.values})
    print('Columns:')
    print(cols, '\n')
    print('Rows: ')
    print(rows, '\n\n')

def categorical_features(df, cols):
    for c in cols:
        print(c)
        print(df[c].value_counts(), '\n')
    print('\n\n')

def get_data():
    #wczytuje dane
    df = pd.read_csv('breast-cancer.data', header=None)
    df.columns = features

    #sprawdzam czy są brakujące wartości, bo w tym zbiorze miały być
    missing_vals(df)
    #plot twist: nie ma

    #patrze jeszcze czy nie ma błędów w zbiorze (np w komórce może być '3' i '3.0', co będzie traktowane jako inne rzeczy, a koresponduje z tą samą wartością) + też brakujące wartości mogły zostać zakodowane jakoś inaczej, tak że isna() ich nie widzi
    categorical_features(df, features)
    #plot twist v2: jest znak zapytania w niektórych xddd

    #wymieniam ? na modę z odpowiadających kolumn, sklearn zwróci mi numpy array, więc konwertuje se z powrotem do dataframea
    imputer = SimpleImputer(missing_values='?', strategy='most_frequent')
    imputed_df = pd.DataFrame(imputer.fit_transform(df))
    imputed_df.columns = df.columns
    imputed_df.index = df.index
    df = imputed_df

    #patrze jeszcze czy sie udało
    categorical_features(df, ['node-caps', 'breast-quad'])

    #dziele sobie, bedziemy przewidywać y na podstawie X
    X, y = df[features[1:]], df[features[0]]

    #nazywam kolumny zeby sie fajniej patrzylo
    X.columns = features[1:]
    y.columns = features[0]

    #rozdzielam zmienne kategoryczne
    X = pd.get_dummies(X, columns=features[1:], drop_first=True)
    y = pd.get_dummies(y, columns=features[0])

    #biore druga kolumne z y tak zeby 1 oznaczalo nawrot, 0 brak nawrotu
    y = y['recurrence-events']

    #zamieniam na numpyowe arraye
    X, y = X.values, y.values

    return X, y

Columns:
   Column name  Missing %
0        class        0.0
1          age        0.0
2    menopause        0.0
3   tumor-size        0.0
4    inv-nodes        0.0
5    node-caps        0.0
6    deg-malig        0.0
7       breast        0.0
8  breast-quad        0.0
9     irradiat        0.0 

Rows: 
   Missing %  Quantity
0        0.0       286 


class
no-recurrence-events    201
recurrence-events        85
Name: class, dtype: int64 

age
50-59    96
40-49    90
60-69    57
30-39    36
70-79     6
20-29     1
Name: age, dtype: int64 

menopause
premeno    150
ge40       129
lt40         7
Name: menopause, dtype: int64 

tumor-size
30-34    60
25-29    54
20-24    50
15-19    30
10-14    28
40-44    22
35-39    19
0-4       8
50-54     8
5-9       4
45-49     3
Name: tumor-size, dtype: int64 

inv-nodes
0-2      213
3-5       36
6-8       17
9-11      10
15-17      6
12-14      3
24-26      1
Name: inv-nodes, dtype: int64 

node-caps
no     222
yes     56
?        8
Name: node-caps,

In [0]:
X, y = get_data()